In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 设置中文字体，确保中文正常显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 验证数据文件是否存在
print("=== 验证文件是否存在 ===")
print(f"当前工作目录: {os.getcwd()}")
print("关键文件是否存在:", 
      os.path.exists("张家口租房合并数据.csv"), 
      os.path.exists("张家口二手房合并数据.csv"))

# 读取租房和二手房数据
print("\n=== 数据读取 ===")
df_rent = pd.read_csv('张家口租房合并数据.csv')
df_sale = pd.read_csv('张家口二手房合并数据.csv')
print(f"租房数据形状: {df_rent.shape}")
print(f"二手房数据形状: {df_sale.shape}")

# 数据清洗和预处理
print("\n=== 数据清洗 ===")

def clean_data(df, data_type):
    """清洗租房或二手房数据，统一格式并计算衍生指标"""
    df_clean = df.copy()
    
    # 确定核心列名
    if data_type == 'rent':
        price_col = '租金(元/月)'
        area_col = '面积'
        region_col = '区域'
    else:
        price_col = '总价'
        area_col = '建筑面积'
        region_col = '区域'
    
    # 去除核心列缺失值
    key_columns = [price_col, area_col, region_col]
    key_columns = [col for col in key_columns if col in df_clean.columns]
    df_clean = df_clean.dropna(subset=key_columns)
    
    # 处理面积列
    df_clean[area_col] = df_clean[area_col].astype(str).str.replace('㎡', '').astype(float)
    
    # 处理价格列
    if data_type == 'rent':
        df_clean[price_col] = pd.to_numeric(df_clean[price_col], errors='coerce')
    else:
        df_clean[price_col] = df_clean[price_col].astype(str).str.replace('万', '').astype(float) * 10000
    
    # 去除无效数据
    df_clean = df_clean[(df_clean[price_col] > 0) & (df_clean[area_col] > 0)]
    
    # 计算每平米价格并统一列名
    df_clean['每平米价格'] = df_clean[price_col] / df_clean[area_col]
    df_clean.rename(columns={region_col: '区域', area_col: '面积'}, inplace=True)
    
    print(f"{data_type}数据清洗完成：有效记录{df_clean.shape[0]}条")
    return df_clean

# 执行数据清洗
df_rent_clean = clean_data(df_rent, 'rent')
df_sale_clean = clean_data(df_sale, 'sale')

# 数据描述和异常值检测
print("\n=== 数据描述和异常值分析 ===")
def describe_with_outliers(df, name):
    """生成数据描述统计量并检测异常值"""
    print(f"\n--- {name}数据描述 ---")
    print(df[['每平米价格', '面积']].describe())
    
    # 用IQR方法检测异常值
    Q1 = df['每平米价格'].quantile(0.25)
    Q3 = df['每平米价格'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df['每平米价格'] < lower_bound) | (df['每平米价格'] > upper_bound)]
    
    print(f"{name}异常值数量: {len(outliers)}")
    return outliers

# 检测并输出异常值信息
rent_outliers = describe_with_outliers(df_rent_clean, "租房")
sale_outliers = describe_with_outliers(df_sale_clean, "二手房")

# 按区域分析价格和租金
print("\n=== 按区域分析 ===")
block_col = '区域'

if block_col in df_rent_clean.columns and block_col in df_sale_clean.columns:
    # 按区域分组计算中位数和样本量
    rent_by_block = df_rent_clean.groupby(block_col)['每平米价格'].agg(['median', 'count']).rename(columns={'median': '租金中位数(元/㎡)'})
    sale_by_block = df_sale_clean.groupby(block_col)['每平米价格'].agg(['median', 'count']).rename(columns={'median': '售价中位数(元/㎡)'})
    
    print("各区域租房数据:")
    print(rent_by_block.round(2))
    print("\n各区域二手房数据:")
    print(sale_by_block.round(2))
    
    # 合并数据计算价格租金比
    merged_data = pd.merge(rent_by_block, sale_by_block, left_index=True, right_index=True, suffixes=('_租', '_售'))
    merged_data['价格租金比'] = merged_data['售价中位数(元/㎡)'] / merged_data['租金中位数(元/㎡)']
    
    print("\n=== 各区域价格租金比 ===")
    print(merged_data[['售价中位数(元/㎡)', '租金中位数(元/㎡)', '价格租金比']].round(0))
    
    # 绘制价格租金比图表
    print("\n=== 绘制价格租金比图表 ===")
    plt.figure(figsize=(12, 8))
    merged_data_sorted = merged_data.sort_values('价格租金比', ascending=False)
    
    bars = plt.bar(merged_data_sorted.index, merged_data_sorted['价格租金比'], color='skyblue', alpha=0.7)
    plt.axhline(y=200, color='red', linestyle='--', label='全球合理值(200)')
    
    # 为柱状图添加数值标签
    for bar, val in zip(bars, merged_data_sorted['价格租金比']):
        plt.text(bar.get_x()+bar.get_width()/2, bar.get_height()+5, f'{val:.0f}', ha='center')
    
    # 设置图表标题和标签
    plt.title('张家口各区域房价租金比', fontsize=16, fontweight='bold')
    plt.xlabel('区域', fontsize=12)
    plt.ylabel('价格租金比', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    plt.savefig('张家口价格租金比分析.png', dpi=300, bbox_inches='tight')
    print("图表已保存为'张家口价格租金比分析.png'")

else:
    print(f"区域列'{block_col}'不存在，当前列名：", df_rent_clean.columns.tolist())

# 生成数据质量报告
print("\n=== 数据质量报告 ===")
print(f"租房数据：原始{df_rent.shape[0]}条 → 有效{df_rent_clean.shape[0]}条（有效率{df_rent_clean.shape[0]/df_rent.shape[0]*100:.1f}%）")
print(f"二手房数据：原始{df_sale.shape[0]}条 → 有效{df_sale_clean.shape[0]}条（有效率{df_sale_clean.shape[0]/df_sale.shape[0]*100:.1f}%）")
    